In [1]:
import chromadb
import uuid
from langchain_community.vectorstores import Chroma
from langchain_ollama import ChatOllama 
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import numpy as np
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
import pickle
import pypdf

In [7]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [4]:
loaders = [
    PyPDFLoader(file_path=r"C:\Users\Usuario\Desktop\Universidad_23_24\TFG\EL_NUEVO\docs\BOCYL-D-17052024-2 2024.pdf", mode="single",),
    PyPDFLoader(r"C:\Users\Usuario\Desktop\Universidad_23_24\TFG\EL_NUEVO\docs\decreto de infantil 30 de octubre 29 DE SEPTIEMBRE.pdf", mode="single"),
    PyPDFLoader(r"C:\Users\Usuario\Desktop\Universidad_23_24\TFG\EL_NUEVO\docs\Decreto primaria octubre 29 SEP 2022.pdf", mode="single"),
    PyPDFLoader(r"C:\Users\Usuario\Desktop\Universidad_23_24\TFG\EL_NUEVO\docs\Decreto secundaria octubre.pdf", mode="single"),
    PyPDFLoader(r"C:\Users\Usuario\Desktop\Universidad_23_24\TFG\EL_NUEVO\docs\Franz_Kafka_The_Metamorphosis.pdf", mode="single")
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

len(docs)

5

In [5]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size= 10000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size= 1000)


In [ ]:
client = chromadb.HttpClient("localhost", 7888)

ampliar_db = True
# coleccion = "prueba_kafka_child"
coleccion = "prueba_kafka_child_augmented"

collection = client.get_or_create_collection(coleccion)

display(client.list_collections())
collection = Chroma(collection_name=coleccion, client=client,  embedding_function=embedding_model, collection_metadata={"hnsw:search_ef": 20})

store = InMemoryStore() 

[Collection(name=ip-coll),
 Collection(name=Spacy),
 Collection(name=prueba_kafka),
 Collection(name=prueba_kafka_large),
 Collection(name=STtoken),
 Collection(name=prueba_kafka_child),
 Collection(name=cosine),
 Collection(name=Recursive),
 Collection(name=multi-qa-mpnet-base-dot-v1),
 Collection(name=prueba_kafka_child_augmented),
 Collection(name=NLTK),
 Collection(name=all-mpnet-base-v2),
 Collection(name=all-MiniLM-L6-v2)]

In [66]:
retriever = ParentDocumentRetriever(
    vectorstore=collection,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [36]:
retriever.add_documents(docs)

In [37]:
llm = ChatOllama(base_url="http://localhost:11434", model="llama3.2:1b")    
prompt = ChatPromptTemplate.from_template("Resume el siguiente texto:\n\n{doc}")

In [ ]:
doc = docs[4]
doc = doc.page_content
query = prompt.invoke(doc)
result = llm.invoke(query)
result
# retriever.vectorstore.add_texts([result.content], [docs[4].metadata])

AIMessage(content='The text appears to be a collection of summaries and excerpts from classic works of literature, including novels, short stories, and poetry. Here are some observations about the text:\n\n1. **Chronology**: The list is organized by decade, with each entry spanning around 11-12 years (1918-1922). This suggests that the author has read a significant number of books from different periods in literature.\n2. **Author selection**: The text highlights several influential authors, including Edgar Allan Poe, Franz Kafka, Gustave Flaubert, and H.P. Lovecraft. These selections suggest an interest in exploring the diversity of literary styles and themes across various centuries.\n3. **Genre representation**: The list covers a range of genres, from Gothic fiction to Modernist literature. This variety highlights the author\'s appreciation for different literary movements and their contributions to the literary canon.\n4. **Publication history**: Some entries mention specific publi

In [69]:
len(docs)


5

In [76]:
if ampliar_db:
    keys = list(store.yield_keys())
    parent_docs = store.mget(keys)
    for doc in parent_docs:
        metadata = doc.metadata
        doc = doc.page_content
        query = prompt.invoke(doc)
        resumen = llm.invoke(query)
        retriever.vectorstore.add_texts([resumen.content], [metadata])

In [ ]:
with open("parent_store.pkl", 'wb') as outp:
    pickle.dump(store, outp, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("parent_store.pkl", 'rb') as inp:
    store = pickle.load(inp)


In [64]:
len(list(store.yield_keys()))

400

In [77]:
np.unique(np.array([ x["source"] for x in collection.get()["metadatas"]])), len(collection.get()["ids"]), collection.get()["data"]

(array(['C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\BOCYL-D-17052024-2 2024.pdf',
        'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\Decreto primaria octubre 29 SEP 2022.pdf',
        'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\Decreto secundaria octubre.pdf',
        'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\Franz_Kafka_The_Metamorphosis.pdf',
        'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\decreto de infantil 30 de octubre 29 DE SEPTIEMBRE.pdf'],
       dtype='<U115'),
 5554,
 None)

In [13]:
#client.delete_collection("prueba_kafka")
# client.create_collection("new_coll", embedding_function=SentenceTransformerEmbeddings("multi-qa-mpnet-base-dot-v1"))


In [14]:
collection.similarity_search("Es aceptable tener una única clase con 35 alumnos de primaria?")

[Document(metadata={'doc_id': 'b72dc07a-32b4-4dc1-9e25-4f9f631a3b4e', 'page': 5, 'source': 'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\BOCYL-D-17052024-2 2024.pdf'}, page_content='1. En virtud de lo dispuesto en el artículo 19.9 del Decreto 38/2022, de 29 de \nseptiembre, cuando el progreso del alumnado no sea el adecuado se establecerán \nmedidas de refuerzo educativo.\nEstas medidas deberán adoptarse en cualquier momento del curso, tan pronto \ncomo se detecten las dificultades y estarán dirigidas a garantizar la adquisición de los'),
 Document(metadata={'doc_id': 'b7cca998-902a-4828-8248-961f52bc7fd8', 'page': 35, 'source': 'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\Decreto primaria octubre 29 SEP 2022.pdf'}, page_content='Boletín Oficial de Castilla y León\nNúm. 190 Pág. 48351Viernes, 30 de septiembre de 2022\nCompetencia personal, social y de aprender a aprender (CPSAA) \nDescriptores operativos: \n \n  \nAl completar la educaci

In [15]:
retriever.invoke("Es aceptable tener una única clase con 35 alumnos de primaria?")

[Document(metadata={'source': 'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\BOCYL-D-17052024-2 2024.pdf', 'page': 5}, page_content='Boletín Oficial de Castilla y León\nNúm. 95 Pág. 22Viernes, 17 de mayo de 2024\nArtículo 9. Medidas de refuerzo educativo, plan específico de refuerzo, plan de \nrecuperación y plan de enriquecimiento curricular.\n1. En virtud de lo dispuesto en el artículo 19.9 del Decreto 38/2022, de 29 de \nseptiembre, cuando el progreso del alumnado no sea el adecuado se establecerán \nmedidas de refuerzo educativo.\nEstas medidas deberán adoptarse en cualquier momento del curso, tan pronto \ncomo se detecten las dificultades y estarán dirigidas a garantizar la adquisición de los \naprendizajes imprescindibles para continuar el proceso educativo.\nSe planificarán por cualquiera de los docentes que atienden al alumno teniendo \nen cuenta las necesidades que se detecten a partir de la información recogida en todo \nmomento acerca del proceso de en

In [16]:
retriever.vectorstore.similarity_search("Primaria")

[Document(metadata={'doc_id': '8057205b-73f4-481b-9886-bccee466b956', 'page': 36, 'source': 'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\Decreto primaria octubre 29 SEP 2022.pdf'}, page_content='Boletín Oficial de Castilla y León\nNúm. 190 Pág. 48352Viernes, 30 de septiembre de 2022\nCompetencia ciudadana (CC) \nDescriptores operativos: \n \n  \nAl completar la educación primaria, el alumno o la alumna…  Al completar la enseñanza básica, el alumno o la alumna…  \nCC1. Entiende los procesos históricos y sociales más \nrelevantes relativos a su propia identidad y cultura,'),
 Document(metadata={'doc_id': '3a05450f-ba4a-4ca8-aa2c-74b272a5b2be', 'page': 44, 'source': 'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\Decreto secundaria octubre.pdf'}, page_content='Boletín Oficial de Castilla y León\nNúm. 190 Pág. 48894Viernes, 30 de septiembre de 2022\nCompetencia ciudadana (CC) \nDescriptores operativos: \n \n  \nAl completar la educación primar

In [ ]:
# col = client.get_or_create_collection("prueba_kafka_child")  # create a new collection with L2 (default)
# newCol = client.get_or_create_collection("prueba_kafka_child_augmented")
# existing_count = col.count()
# batch_size = 10
# for i in range(0, existing_count, batch_size):
#     batch = col.get(include=["metadatas", "documents", "embeddings"], limit=batch_size, offset=i)
#     newCol.add(ids=batch["ids"], documents=batch["documents"], metadatas=batch["metadatas"],
#                embeddings=batch["embeddings"])

# print(newCol.count())
# print(newCol.get(offset=0, limit=10))  # get first 10 documents


5149
{'ids': ['7b7fced6-b381-4c66-a54f-e690aa5de810', '9db643d6-fb24-41be-8cd2-15242fbeab83', '621386ec-83d1-45da-992b-3698b1c41026', '27fd5a48-869e-4c76-86da-31376a171784', 'cb7db874-523d-4095-b711-087f16a145fd', '06380e31-ad5f-4b3b-880f-5c69c16c7c7c', '61db5ef8-13c2-4e60-b40c-de8c501486fa', 'fa662d5d-8c98-43f5-9dbe-155345a05c21', 'bcf1d8bd-8e5a-41a1-ae2f-2a44837240c1', '85af14c9-b807-4c3e-bf11-c000b62878e3'], 'embeddings': None, 'metadatas': [{'author': 'JUNTA DE CASTILLA Y LEÓN', 'creationdate': '2024-05-16T13:54:42+02:00', 'creator': 'Adobe InDesign 19.4 (Windows)', 'doc_id': '2ad7db1f-622c-4719-97c3-53406f09e025', 'moddate': '2024-05-17T07:30:24+02:00', 'producer': 'Adobe PDF Library 17.0; modified using iText 2.1.7 by 1T3XT', 'source': 'C:\\Users\\Usuario\\Desktop\\Universidad_23_24\\TFG\\EL_NUEVO\\docs\\BOCYL-D-17052024-2 2024.pdf', 'title': 'BOCyL n.º 95,  17 de mayo de 2024 - Disp. 002', 'total_pages': 49, 'trapped': '/False'}, {'author': 'JUNTA DE CASTILLA Y LEÓN', 'creationd